## Matrix and Vocabulary Construction

# Código feito pelos monitores.

In [1]:
import pandas as pd

import numpy as np

from scipy import sparse

import nltk
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer

import math
__author__ = "Thierry Barros"

indice_invertido = {}
dados_TF = {}



In [2]:
news = pd.read_csv("../data/estadao_noticias_eleicao.csv", encoding="utf-8")

In [3]:
content = news.titulo + " " + news.subTitulo + " " + news.conteudo
content = content.fillna("")

In [4]:
def co_occurrence_matrix(corpus):
    vocab = set(corpus)
    vocab = list(vocab)
    n = len(vocab)
   
    vocab_to_index = {word:i for i, word in enumerate(vocab)}
    
    bi_grams = list(bigrams(corpus))

    bigram_freq = nltk.FreqDist(bi_grams).most_common(len(bi_grams))

    I=list()
    J=list()
    V=list()
    
    for bigram in bigram_freq:
        current = bigram[0][1]
        previous = bigram[0][0]
        count = bigram[1]

        I.append(vocab_to_index[previous])
        J.append(vocab_to_index[current])
        V.append(count)
        
    co_occurrence_matrix = sparse.coo_matrix((V,(I,J)), shape=(n,n))

    return co_occurrence_matrix, vocab_to_index

#### Removing punctuation

In [5]:
tokenizer = RegexpTokenizer(r'\w+')
tokens_lists = content.apply(lambda text: tokenizer.tokenize(text.lower()))

#### Removing stopwords

In [6]:
stopword_ = stopwords.words('portuguese')
filtered_tokens = tokens_lists.apply(lambda tokens: [token for token in tokens if token not in stopword_])

#### Transforming list of lists into one list

In [7]:
tokens = [token for tokens_list in filtered_tokens for token in tokens_list]

In [8]:
matrix, vocab = co_occurrence_matrix(tokens)

## Consult Bigram Frequency

In [9]:
consultable_matrix = matrix.tocsr()

In [10]:
def consult_frequency(w1, w2):
    return(consultable_matrix[vocab[w1],vocab[w2]])

# Código do lab 1 para pesquisa OR.

In [11]:
def preencher( tupla ):
    palavra, id_texto = tupla
    if palavra.lower() not in dados_TF[id_texto]:
        dados_TF[id_texto][palavra.lower()] = 0
    dados_TF[id_texto][palavra.lower()] += 1
    try:
        indice_invertido [ palavra.lower() ].add(id_texto)
    except KeyError:
        indice_invertido[ palavra.lower() ] = set([ id_texto ])

In [12]:

def extrair( texto , id_texto ):
    for i in xrange( len(texto) ):
        dados_TF[id_texto[i]]= {}
        for palavra in nltk.word_tokenize( str(texto[i]).decode('utf-8') ):
            preencher((palavra,id_texto[i]))



#Faça uma busca disjuntiva (OR) considerando a nova consulta.

**Função de busca OR Binário.**
Função de busca que retorna os documentos que possuem pelo menos uma das palavras da consulta. Função de labs anteriores é uma função de busca binária.

In [13]:
def procurarOR(lista):
    indices = indice_invertido[lista[0].lower()]
    for palavra in lista:
        indices = set(reduce( lambda x,y : list( x | y ), [ indice_invertido[palavra.lower()] , indices] ))
    return indices

**Função main** que roda o programa e cria o indíce invertido.

In [14]:

file_csv = pd.read_csv("../data/estadao_noticias_eleicao.csv")
file_csv = file_csv.replace(np.nan, '', regex=True)
texto = file_csv.titulo + " " + file_csv.conteudo + " " + file_csv.subTitulo
extrair(texto,file_csv.idNoticia)

# Código do Lab2 - Expansão de Consultas.

#Escreva uma função que receba um certo termo de consulta e a matriz construída no passo 1 acima e retorne as top-3 palavras em ordem decrescente de frequencia.

A função pega os tokens(todas as palavras do dicionário), faz um set para tirar as repetições, depois tranforma e lista denovo. Então eu crio uma lista(dic), contendo tuplas de (palavra, frequência da palavra com o termo). Após a criação dessa lista, eu ordeno ela pela frequência, e pego as três palavras com maior frequência e retorno elas. 
OBS: Crio essa lista de nome, frequencia, ultilizando a função auxiliar consult_frequency(word, i), criada pelos monitores da turma, então eu não preciso receber na função a matrix de co-ocorrência, porque essa função já usa a matriz.

In [15]:
def top3_similarity(word):
    words = set(tokens)
    words = list(words)
    dic = []
    for i in words:
        try:
            dic.append((i, consult_frequency(word, i)))
        except KeyError:
            return ["Not Found: palavra '"+word+"' não encontrada"]
    dic.sort(key=lambda tup: tup[1], reverse=True)
    similar_words = []
    for tupla in dic[:3]:
       similar_words.append(tupla[0])
    return similar_words

#Expanda a consulta original com os termos retornados no passo 2 acima.

Essa função recebe o termo a ser procurado e retorna a lista expandida com os top3 palavras mais similares.

In [16]:
def expand_consult(word):
    words = []
    words.append(word)
    words += top3_similarity(word)
    return words

##Escolha livremente três termos de consulta e responda o seguinte:

**Os três termos escolhidos foram: Poucos, recursos, corinthians.**

#Quais os termos retornados para a expansão de cada consulta?

In [30]:
word1 = "dilma"
word2 = "recursos"
word3 = "corinthians"

ex_word1 = expand_consult(word1)

ex_word2 = expand_consult(word2)

ex_word3 = expand_consult(word3)


#Consulta **dilma**:

In [31]:
print("Termos encontrados na expansão da pesquisa de "+word1+":")
for i, word in enumerate(ex_word1):
    print i+1,word

Termos encontrados na expansão da pesquisa de dilma:
1 dilma
2 rousseff
3 é
4 disse


**Você acha que esses termos são de fato relacionados com a consulta original? Justifique.**

Sim, pois são termos que em conjunto fazem sentido. Exemplo: Dilma Rousseff que é o sobre nome dela, dilma é, dilma disse(já que ela gosta de fazer várias discursos e declarações polêmicas).

#Consulta **recursos**:

In [32]:
print("Termos encontrados na expansão da pesquisa de "+word2+":")
for i, word in enumerate(ex_word2):
    print i+1,word

Termos encontrados na expansão da pesquisa de recursos:
1 recursos
2 públicos
3 fundo
4 petrobrás


**Você acha que esses termos são de fato relacionados com a consulta original? Justifique.**

Sim, mas acho que só duas das palavras fazem sentido, que são públicos e petrobrás. Recursos fundo não parece fazer muito sentido em conjunto, mas pode ser porque a frase não está completa, poderia ter outras palavras após essas que na frase fariam mais sentido. Já, recursos públicos e recursos petrobrás parece fazer sentido, deve possivelmente está relacionado ao interesse dos usuários em saber como estão sendo gastos os recursos do país.

#Consulta **corinthians**:

In [33]:
print("Termos encontrados na expansão da pesquisa de "+word3+":")
for i, word in enumerate(ex_word3):
    print i+1,word

Termos encontrados na expansão da pesquisa de corinthians:
1 corinthians
2 andrés
3 2010
4 paulo


**Você acha que esses termos são de fato relacionados com a consulta original? Justifique.**

Sim, pois Andrés é o presidente do timão. 2010 deve ter sido a pesquisa em alta no ano da captação desses dados, procurando saber como o corinthians foi ou que jogos ele teria no ano de 2010. Paulo pode ser uma palavra ambígua, oque pode trazer problemas com falsos positivos caso seja uma palavra estatísticamente irrelevante, mas ela tanto pode representar o estado de São paulo, ou algum jogador chamado Paulo que fez sucesso no corinthians na época, confesso que não lembro de nenhum jogador com esse nome.

**Compare os documentos retornados para a consulta original com a consulta expandida. Quais resultados você acha que melhor capturam a necessidade de informação do usuário? Por que?**

Para responder essa questão vou pegar **5 documentos** da consulta com o **termo original** e da consulta **expandida** e olhar manualmente quais documentos de quais pesquisas são **mais relevantes para a consulta original**.

In [34]:
file_csv = pd.read_csv("../data/estadao_noticias_eleicao.csv")
file_csv = file_csv.replace(np.nan, '', regex=True)
titulo = file_csv.titulo

#Documentos da consulta **poucos**.

In [44]:
documentos = list(procurarOR([word1]))[10:15]
documentos_extendidos = list(procurarOR(ex_word1))[10:15]
print("Documentos retornados pela consulta original:\n")
for i,indice in enumerate(documentos):
    print i+1,titulo[indice]
print("\nDocumentos retornados pela consulta expandida:\n")
for i,indice in enumerate(documentos_extendidos):
    print i+1,titulo[indice]

Documentos retornados pela consulta original:

1 Dilma anuncia últimos 14 ministros e fecha equipe
2 Ex-governador critica Alckmin pela falta de água em SP
3 Veja lista completa de ministros anunciados por Dilma
4 'PT não pode se queixar', afirma futuro articulador  
5 Planalto confirma Juca Ferreira no Ministério da Cultura

Documentos retornados pela consulta expandida:

1 Planalto confirma Juca Ferreira no Ministério da Cultura
2 Pimentel divulga nomes do secretariado do governo de Minas
3 Ala do PT pede saída de Ideli de ministério
4 Cid Gomes anunciará novo piso para professores em janeiro
5 Além de ministério, PRB ganha pastas estaduais de Esporte   


No caso da pesquisa por Dilma, como é um nome bem incomum a pesquisa expandida não parece ter trazido documentos mais relevantes, isso claro se baseando apenas no título de algumas notícias, oque não serve de prova e é muito simples para dizer que é a algo verídico, mas as duas buscas trouxe documentos relacionados com a pesquisa.

#Documentos da consulta **recursos**.

In [45]:
documentos = list(procurarOR([word2]))[5:10]
documentos_extendidos = list(procurarOR(ex_word2))[5:10]
print("Documentos retornados pela consulta original:\n")
for i,indice in enumerate(documentos):
    print i+1,titulo[indice]
print("\nDocumentos retornados pela consulta expandida:\n")
for i,indice in enumerate(documentos_extendidos):
    print i+1,titulo[indice]

Documentos retornados pela consulta original:

1 TRE-RJ nega registro de Cesar Maia para concorrer ao Senado
2 Aécio anuncia subdivisão em programa de transferência
3 Com recusa de Celso Amorim, Figueiredo deve ficar no Itamaraty 
4 A superfície e o subsolo das eleições
5 STJ julga recurso do coronel Ustra

Documentos retornados pela consulta expandida:

1 PT impulsiona cerimônia de posse da Dilma nas redes sociais
2 PSB-Rede apresenta diretrizes do programa de governo
3 Veja lista completa de ministros anunciados por Dilma
4 Planalto confirma Juca Ferreira no Ministério da Cultura
5 Se essa rua fosse minha


Já no caso da pesquisa por **recursos** a pesquisa expandida e a original também parecem ter trazido documentos com o mesmo nível de relevância e muito relacionado a recursos políticos e econômicos, mas lembrando que é uma análise bem subjetiva e simples.

#Documentos da consulta **corinthians**.

In [46]:
documentos = list(procurarOR([word3]))[16:21]
documentos_extendidos = list(procurarOR(ex_word3))[16:21]
print("Documentos retornados pela consulta original:\n")
for i,indice in enumerate(documentos):
    print i+1,titulo[indice]
print("\nDocumentos retornados pela consulta expandida:\n")
for i,indice in enumerate(documentos_extendidos):
    print i+1,titulo[indice]

Documentos retornados pela consulta original:

1 PSDB organiza maratona de eventos pró-Aécio em MG
2 Ibope aponta empate técnico entre Jatene e Helder Barbalho no PA
3 Mistura de estações  
4 Alckmin nomeia “homem de confiança” de Russomanno (PRB) para a direção do Procon
5 Eunicio Oliveira será relator do projeto de terrorismo no Senado e ganha vitrine para disputar governo do Ceará

Documentos retornados pela consulta expandida:

1 Menor prestígio e verba faz diplomatas buscarem opções a carreira no Itamaraty  
2 Em carta, João Paulo diz que vai se entregar mas não vai renunciar
3 Cúpula da política externa proíbe uso de Facebook para queixas  
4 Dilma tenta reduzir poder de PT e PMDB
5 Executiva de banco  diz na Justiça que PT 'exigiu' sua demissão  


As buscas por corinthians na consulta original, pelo menos pelo título não parecem ter muita relação. Já alguns títulos de textos da busca expandida parecem ter mais relação com a busca, mostrando que em alguns casos a busca expandida pode melhorar o número de documentos relevantes retornados.

#A expansão de consultas é mais adequada para melhorar o recall ou o precision? Por que?

Para responder essa questão vou checar o número de documentos retornados pelas duas consultas.

In [38]:
print "N° documentos retornado pela consulta "+word1+":"+str(len(procurarOR([word1])))
print "N° documentos retornado pela consulta expandida "+str(ex_word1)+":"+str(len(procurarOR(ex_word1)))

N° documentos retornado pela consulta dilma:4269
N° documentos retornado pela consulta expandida ['dilma', u'rousseff', u'\xe9', u'disse']:7396


In [42]:
print "N° documentos retornado pela consulta "+word2+":"+str(len(procurarOR([word2])))
print "N° documentos retornado pela consulta expandida "+str(ex_word2)+":"+str(len(procurarOR(ex_word2)))

N° documentos retornado pela consulta recursos:1025
N° documentos retornado pela consulta expandida ['recursos', u'p\xfablicos', u'fundo', u'petrobr\xe1s']:2520


In [43]:
print "N° documentos retornado pela consulta "+word3+":"+str(len(procurarOR([word3])))
print "N° documentos retornado pela consulta expandida "+str(ex_word3)+":"+str(len(procurarOR(ex_word3)))

N° documentos retornado pela consulta corinthians:22
N° documentos retornado pela consulta expandida ['corinthians', u'andr\xe9s', u'2010', u'paulo']:3934


Podemos ver que os 3 casos o número de documentos retornados pelas consultas expandidas é bem maior que o número retornado pela consulta só com a palavra original.Isso sempre vai acontecer já que a consulta expandida contém o termo da consulta original e é uma busca por OR, então **no mínimo a consulta expandida trará o mesmo número de documentos que a consulta original**, como podemos ver em geral ela trará mais documentos. Com base nisso, em relação ao **recall**(N° de documentos relevantes retornados dentre todos os relevantes), então podemos ver que o **recall da consulta expandida sempre será maior ou igual ao recall da consulta original**. No entando **sobre a precision não podemos afirmar nada com certeza**, pode ser um trade-off ao mesmo tempo que aumentasmo o recall, aumentando o número de documentos retornados, podemos está diminuindo, ou não, o precision da consulta. Não sabemos quantos dos novos documentos retornados são relevantes para a consulta. Como a consulta binária não faz um raking dos documentos, é difícil afirmar se os documentos são relevantes ou não, mas por eurísticas e testes já feitos anteriormente por pesquisadores da área, foi mostrado que na maioria dos casos a busca expandida melhora o retorno da consulta.